# Libraries

In [168]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import os
import re
import random
import itertools
import warnings
warnings.simplefilter("ignore")

from sklearn import preprocessing

import dgl
from dgl.data import DGLDataset
import dgl.nn as dglnn
from dgl.nn import GraphConv, SAGEConv, GATConv
import dgl.function as fn

import torch
import torch.nn as nn
import torch.nn.functional as F

import pygraphviz as pgv

import scipy.sparse as sp

In [169]:
n_input_feat = 10
n_hidden_feat = 20
n_output_feat = 10

k = 10 # for negative graph: each edge gets k negative examples - 'negative sampling'

In [170]:
vModel = 'GCN'

In [171]:
if vModel == 'GCN':
    n_epochs = 201
elif vModel == 'SAGE':
    n_epochs = 601

# 1) Create DGL Heterograph

## a. Get KG

In [172]:
f = open('../Input Data/statements.nq', 'r')
text = f.readlines()

In [173]:
rows = []
for line in text:
  split = line.split()
  s = split[0]
  s = s.replace('<', '')
  s = s.replace('>', '')
  p = split[1]
  p = p.replace('<', '')
  p = p.replace('>', '')
  o = split[2]
  o = o.replace('<', '')
  o = o.replace('>', '')
  rows.append([s, p, o])

In [174]:
triples = pd.DataFrame(rows, columns=['subject', 'predicate', 'object'])
triples.head()

,subject,predicate,object
0,http://idea.rpi.edu/heals/kb/usda#01003,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://idea.rpi.edu/heals/kb/usda
1,http://idea.rpi.edu/heals/kb/usda#01004,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://idea.rpi.edu/heals/kb/usda
2,http://idea.rpi.edu/heals/kb/usda#01005,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://idea.rpi.edu/heals/kb/usda
3,http://idea.rpi.edu/heals/kb/usda#01006,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://idea.rpi.edu/heals/kb/usda
4,http://idea.rpi.edu/heals/kb/usda#01007,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://idea.rpi.edu/heals/kb/usda


In [175]:
print(f'# Triples: {len(triples)}')

# Triples: 3022948


## b. Convert KG to pd.DataFrame

In [176]:
all_foods = triples[(triples['subject'].str.startswith('http://idea.rpi.edu/heals/kb/usda#'))]
le_food = preprocessing.LabelEncoder()
le_food.fit(all_foods['subject'].tolist())

LabelEncoder()

In [177]:
def map_value_2_id(df, value, name):
    le = preprocessing.LabelEncoder()
    le.fit(df[value].tolist())
    df[name] = le.transform(df[value].tolist())
    return df, le

**(Food, containsNutrient, Nutrient)**

In [178]:
df_food_nutrient = triples[(triples['subject'].str.startswith('http://idea.rpi.edu/heals/kb/usda#')) &
                        (triples['predicate'] == 'http://www.w3id.org/foodkg/property#contains') &
                        (triples['object'].str.startswith('http://www.w3id.org/foodkg/usda#'))]   

df_food_nutrient['source_id'] = le_food.transform(df_food_nutrient['subject'].tolist())
df_food_nutrient, le_nutrient = map_value_2_id(df_food_nutrient, 'object', 'destination_id')

**(Food, hasTag, Tag)**

In [179]:
df_food_tag = triples[(triples['subject'].str.startswith('http://idea.rpi.edu/heals/kb/usda#')) &
                      (triples['predicate'] == 'http://www.w3id.org/foodb/property/hasQuality') &
                      (triples['object'].str.startswith('https://w3id.org/foodkg/quality/high_'))]   

df_food_tag['source_id'] = le_food.transform(df_food_tag['subject'].tolist())
df_food_tag, le_tag = map_value_2_id(df_food_tag, 'object', 'destination_id')

**(Food, isInCategory, Category)**

In [180]:
df_food_cat = triples[(triples['subject'].str.startswith('http://idea.rpi.edu/heals/kb/usda#')) &
                      (triples['predicate'] == 'http://www.w3id.org/foodb/property/hasCategory') &
                      (triples['object'].str.startswith('http://idea.rpi.edu/heals/kb/usda#'))]   

df_food_cat['source_id'] = le_food.transform(df_food_cat['subject'].tolist())
df_food_cat, le_category = map_value_2_id(df_food_cat, 'object', 'destination_id')

**(Food, hasFlavor, Flavor)**

In [181]:
df_food_flavor = triples[(triples['subject'].str.startswith('http://idea.rpi.edu/heals/kb/usda#')) &
                      (triples['predicate'] == 'http://www.w3id.org/foodb/property/hasFlavor') &
                      (triples['object'].str.startswith('https://w3id.org/foodkg/flavor/'))]   

df_food_flavor['source_id'] = le_food.transform(df_food_flavor['subject'].tolist())
df_food_flavor, le_flavor = map_value_2_id(df_food_flavor, 'object', 'destination_id')

**(Product, containsIngredient, Ingredient)**

In [182]:
df_product_ingredient = triples[(triples['subject'].str.startswith('https://w3id.org/um/ken4256/product/')) &
                       (triples['predicate'] == 'https://www.bbc.co.uk/ontologies/fo/ingredients') &
                       (triples['object'].str.startswith('https://w3id.org/um/ken4256/ingredient/'))]   

df_product_ingredient, le_product = map_value_2_id(df_product_ingredient, 'subject', 'source_id')
df_product_ingredient, le_ingredient = map_value_2_id(df_product_ingredient, 'object', 'destination_id')

**(Food, isSimilarTo, Ingredient) - (Ingredient, isSimilarTo, Food)**

In [183]:
df_food_ingredient = triples[(triples['subject'].str.startswith('http://idea.rpi.edu/heals/kb/usda#')) &
                      (triples['predicate'] == 'https://schema.org/isSimilarTo') &
                      (triples['object'].str.startswith('https://w3id.org/um/ken4256/ingredient/'))]   

df_food_ingredient['source_id'] = le_food.transform(df_food_ingredient['subject'].tolist())
df_food_ingredient['destination_id'] = le_ingredient.transform(df_food_ingredient['object'].tolist())

## c. Get Ground Truth and convert to pd.DataFrame

**(Food, isSubstitutedBy, Food Subs) - (Food Subs, canSubstitute, Food)**

In [184]:
df_food_subs = pd.read_csv('../Input Data/final_substitution.csv', sep=';')

df_food_subs['source_id'] = le_food.transform(df_food_subs['Food id'].tolist())
df_food_subs['destination_id'] = le_food.transform(df_food_subs['Substitution id'].tolist())

## d. Create DGL HeteroGraph

In [185]:
g = dgl.heterograph({('Food', 'isSubstitutedBy', 'Food'): (torch.tensor(df_food_subs['source_id'].tolist()), torch.tensor(df_food_subs['destination_id'].tolist())),
                     ('Food', 'substitutes', 'Food'): (torch.tensor(df_food_subs['destination_id'].tolist()), torch.tensor(df_food_subs['source_id'].tolist())),
                     ('Food', 'containsNutrient', 'Nutrient'): (torch.tensor(df_food_nutrient['source_id'].tolist()), torch.tensor(df_food_nutrient['destination_id'].tolist())),
                     ('Food', 'hasTag', 'Tag'): (torch.tensor(df_food_tag['source_id'].tolist()), torch.tensor(df_food_tag['destination_id'].tolist())),
                     ('Food', 'isInCategory', 'Category'): (torch.tensor(df_food_cat['source_id'].tolist()), torch.tensor(df_food_cat['destination_id'].tolist())),
                     ('Food', 'hasFlavor', 'Flavor'): (torch.tensor(df_food_flavor['source_id'].tolist()), torch.tensor(df_food_flavor['destination_id'].tolist())),
                     ('Product', 'containsIngredient', 'Ingredient'): (torch.tensor(df_product_ingredient['source_id'].tolist()), torch.tensor(df_product_ingredient['destination_id'].tolist())),
                     ('Food', 'sameAs', 'Ingredient'): (torch.tensor(df_food_ingredient['source_id'].tolist()), torch.tensor(df_food_ingredient['destination_id'].tolist())),
                     ('Ingredient', 'sameAs', 'Food'): (torch.tensor(df_food_ingredient['destination_id'].tolist()), torch.tensor(df_food_ingredient['source_id'].tolist()))
                    })

In [186]:
for node in g.ntypes:
    g.nodes[node].data['h'] = torch.randn(g.number_of_nodes(node), n_input_feat)

In [187]:
g

Graph(num_nodes={'Category': 13, 'Flavor': 272, 'Food': 9372, 'Ingredient': 125130, 'Nutrient': 63883, 'Product': 71777, 'Tag': 25},
      num_edges={('Food', 'containsNutrient', 'Nutrient'): 300523, ('Food', 'hasFlavor', 'Flavor'): 11167, ('Food', 'hasTag', 'Tag'): 17746, ('Food', 'isInCategory', 'Category'): 1667, ('Food', 'isSubstitutedBy', 'Food'): 1841, ('Food', 'sameAs', 'Ingredient'): 681, ('Food', 'substitutes', 'Food'): 1841, ('Ingredient', 'sameAs', 'Food'): 681, ('Product', 'containsIngredient', 'Ingredient'): 890789},
      metagraph=[('Food', 'Nutrient', 'containsNutrient'), ('Food', 'Flavor', 'hasFlavor'), ('Food', 'Tag', 'hasTag'), ('Food', 'Category', 'isInCategory'), ('Food', 'Food', 'isSubstitutedBy'), ('Food', 'Food', 'substitutes'), ('Food', 'Ingredient', 'sameAs'), ('Ingredient', 'Food', 'sameAs'), ('Product', 'Ingredient', 'containsIngredient')])

In [188]:
g.number_of_nodes('Food')

9372

In [189]:
g.number_of_edges('isSubstitutedBy')

1841

In [190]:
g.ntypes

['Category', 'Flavor', 'Food', 'Ingredient', 'Nutrient', 'Product', 'Tag']

In [191]:
g.etypes

['containsNutrient',
 'hasFlavor',
 'hasTag',
 'isInCategory',
 'isSubstitutedBy',
 'sameAs',
 'substitutes',
 'sameAs',
 'containsIngredient']

In [192]:
g.canonical_etypes

[('Food', 'containsNutrient', 'Nutrient'),
 ('Food', 'hasFlavor', 'Flavor'),
 ('Food', 'hasTag', 'Tag'),
 ('Food', 'isInCategory', 'Category'),
 ('Food', 'isSubstitutedBy', 'Food'),
 ('Food', 'sameAs', 'Ingredient'),
 ('Food', 'substitutes', 'Food'),
 ('Ingredient', 'sameAs', 'Food'),
 ('Product', 'containsIngredient', 'Ingredient')]

In [193]:
def plot_graph(nxg):
    ag = pgv.AGraph(strict = False, directed = True)
    for u, v, k in nxg.edges(keys=True):
        ag.add_edge(u, v, label=k)
    ag.layout('dot')
    ag.draw('../Output/graph.png')

In [194]:
plot_graph(g.metagraph())

## e. Split into train and test sets

In [195]:
#randomly generate training masks on 'isSubstitutedBy' edges
g.edges['isSubstitutedBy'].data['train_mask'] = torch.zeros(g.number_of_edges('isSubstitutedBy'), dtype=torch.bool).bernoulli(0.75)

In [196]:
g.edges['isSubstitutedBy']

EdgeSpace(data={'train_mask': tensor([True, True, True,  ..., True, True, True])})

In [197]:
train_eid_dict = {'eid_2_train': (g.edges['isSubstitutedBy'].data['train_mask'] == True).nonzero(as_tuple=True)[0]}
test_eid_dict = {'eid_2_test': (g.edges['isSubstitutedBy'].data['train_mask'] == False).nonzero(as_tuple=True)[0]}

In [198]:
train_eid_dict['eid_2_train']

tensor([   0,    1,    2,  ..., 1838, 1839, 1840])

In [199]:
test_eid_dict['eid_2_test']

tensor([  11,   18,   19,   24,   25,   33,   34,   35,   36,   37,   40,   41,
          42,   46,   60,   61,   62,   65,   67,   68,   71,   72,   74,   77,
          79,   80,   88,   96,   99,  103,  104,  106,  107,  117,  128,  130,
         139,  142,  143,  145,  146,  147,  151,  152,  158,  161,  169,  173,
         174,  176,  180,  185,  187,  199,  210,  215,  226,  228,  235,  238,
         241,  243,  253,  254,  258,  259,  260,  262,  269,  275,  285,  286,
         290,  294,  302,  305,  306,  311,  314,  316,  317,  318,  322,  323,
         325,  329,  332,  335,  337,  342,  349,  353,  355,  357,  358,  361,
         362,  365,  368,  370,  373,  374,  377,  380,  386,  387,  388,  391,
         394,  396,  398,  403,  405,  410,  412,  413,  414,  416,  418,  419,
         420,  429,  430,  432,  436,  445,  446,  451,  453,  454,  455,  456,
         461,  465,  466,  470,  474,  476,  481,  482,  485,  486,  488,  490,
         493,  501,  503,  507,  513,  5

In [200]:
g.number_of_edges('isSubstitutedBy')

1841

In [201]:
len(train_eid_dict['eid_2_train'])

1391

In [202]:
len(test_eid_dict['eid_2_test'])

450

In [203]:
g_train = dgl.remove_edges(g, test_eid_dict['eid_2_test'], 'isSubstitutedBy')
g_train = dgl.remove_edges(g_train, test_eid_dict['eid_2_test'], 'substitutes')

In [204]:
g_train

Graph(num_nodes={'Category': 13, 'Flavor': 272, 'Food': 9372, 'Ingredient': 125130, 'Nutrient': 63883, 'Product': 71777, 'Tag': 25},
      num_edges={('Food', 'containsNutrient', 'Nutrient'): 300523, ('Food', 'hasFlavor', 'Flavor'): 11167, ('Food', 'hasTag', 'Tag'): 17746, ('Food', 'isInCategory', 'Category'): 1667, ('Food', 'isSubstitutedBy', 'Food'): 1391, ('Food', 'sameAs', 'Ingredient'): 681, ('Food', 'substitutes', 'Food'): 1391, ('Ingredient', 'sameAs', 'Food'): 681, ('Product', 'containsIngredient', 'Ingredient'): 890789},
      metagraph=[('Food', 'Nutrient', 'containsNutrient'), ('Food', 'Flavor', 'hasFlavor'), ('Food', 'Tag', 'hasTag'), ('Food', 'Category', 'isInCategory'), ('Food', 'Food', 'isSubstitutedBy'), ('Food', 'Food', 'substitutes'), ('Food', 'Ingredient', 'sameAs'), ('Ingredient', 'Food', 'sameAs'), ('Product', 'Ingredient', 'containsIngredient')])

In [205]:
g_train.number_of_edges('isSubstitutedBy')

1391

In [206]:
g_train.number_of_edges('substitutes')

1391

In [207]:
g_test = dgl.remove_edges(g, train_eid_dict['eid_2_train'], 'isSubstitutedBy')
g_test = dgl.remove_edges(g_test, train_eid_dict['eid_2_train'], 'substitutes')

In [208]:
g_test

Graph(num_nodes={'Category': 13, 'Flavor': 272, 'Food': 9372, 'Ingredient': 125130, 'Nutrient': 63883, 'Product': 71777, 'Tag': 25},
      num_edges={('Food', 'containsNutrient', 'Nutrient'): 300523, ('Food', 'hasFlavor', 'Flavor'): 11167, ('Food', 'hasTag', 'Tag'): 17746, ('Food', 'isInCategory', 'Category'): 1667, ('Food', 'isSubstitutedBy', 'Food'): 450, ('Food', 'sameAs', 'Ingredient'): 681, ('Food', 'substitutes', 'Food'): 450, ('Ingredient', 'sameAs', 'Food'): 681, ('Product', 'containsIngredient', 'Ingredient'): 890789},
      metagraph=[('Food', 'Nutrient', 'containsNutrient'), ('Food', 'Flavor', 'hasFlavor'), ('Food', 'Tag', 'hasTag'), ('Food', 'Category', 'isInCategory'), ('Food', 'Food', 'isSubstitutedBy'), ('Food', 'Food', 'substitutes'), ('Food', 'Ingredient', 'sameAs'), ('Ingredient', 'Food', 'sameAs'), ('Product', 'Ingredient', 'containsIngredient')])

In [209]:
g_test.number_of_edges('isSubstitutedBy')

450

In [210]:
g_test.number_of_edges('substitutes')

450

# 2) Train Model

**Hetero-GCN**

In [211]:
class HGCN(nn.Module):
    def __init__(self, in_feats, hid_feats, out_feats, rel_names):
        super().__init__()

        self.conv1 = dglnn.HeteroGraphConv({
            rel: dglnn.GraphConv(in_feats, hid_feats)
            for rel in rel_names}, aggregate='sum')
        self.conv2 = dglnn.HeteroGraphConv({
            rel: dglnn.GraphConv(hid_feats, out_feats)
            for rel in rel_names}, aggregate='sum')

    def forward(self, graph, inputs):
        # inputs are features/embeddings of nodes
        h = self.conv1(graph, inputs)
        h = {k: F.relu(v) for k, v in h.items()}
        h = self.conv2(graph, h)
        return h

In [212]:
class ModelHGCN(nn.Module):
    def __init__(self, in_features, hidden_features, out_features, rel_names):
        super().__init__()
        self.sage = HGCN(in_features, hidden_features, out_features, rel_names)
        self.pred = HeteroDotProductPredictor()
    def forward(self, g, neg_g, x, etype):
        h = self.sage(g, x)
        return h, self.pred(g, h, etype), self.pred(neg_g, h, etype)

**Hetero-SAGE**

In [213]:
class HSAGE(nn.Module):
    def __init__(self, in_feats, hid_feats, out_feats, rel_names):
        super().__init__()

        self.conv1 = dglnn.HeteroGraphConv({
            rel: dglnn.SAGEConv(in_feats, hid_feats, 'mean')
            for rel in rel_names}, aggregate='sum')
        self.conv2 = dglnn.HeteroGraphConv({
            rel: dglnn.SAGEConv(hid_feats, out_feats, 'mean')
            for rel in rel_names}, aggregate='sum')

    def forward(self, graph, inputs):
        # inputs are features/embeddings of nodes
        h = self.conv1(graph, inputs)
        h = {k: F.relu(v) for k, v in h.items()}
        h = self.conv2(graph, h)
        return h

In [214]:
class ModelHSAGE(nn.Module):
    def __init__(self, in_features, hidden_features, out_features, rel_names):
        super().__init__()
        self.sage = HSAGE(in_features, hidden_features, out_features, rel_names)
        self.pred = HeteroDotProductPredictor()
    def forward(self, g, neg_g, x, etype):
        h = self.sage(g, x)
        return h, self.pred(g, h, etype), self.pred(neg_g, h, etype)

In [215]:
def construct_negative_graph(graph, k, etype):
    utype, _, vtype = etype
    src, dst = graph.edges(etype=etype)
    neg_src = src.repeat_interleave(k)
    neg_dst = torch.randint(0, graph.number_of_nodes(vtype), (len(src) * k,))
    return neg_src, neg_dst, dgl.heterograph({etype: (neg_src, neg_dst)}, num_nodes_dict={ntype: graph.number_of_nodes(ntype) for ntype in graph.ntypes})

In [216]:
class HeteroDotProductPredictor(nn.Module):
    def forward(self, graph, h, etype):
        with graph.local_scope():
            graph.ndata['h'] = h
            graph.apply_edges(fn.u_dot_v('h', 'h', 'score'), etype=etype)
            return graph.edges[etype].data['score']

In [217]:
def compute_loss(pos_score, neg_score):
    n_edges = pos_score.shape[0]
    return (1 - neg_score.view(n_edges, -1) + pos_score.unsqueeze(1)).clamp(min=0).mean()

In [218]:
node_features = {}
for node in g.ntypes:
    node_features[node] = g.nodes[node].data['h'] 

In [219]:
if vModel == 'GCN':
    model = ModelHGCN(n_input_feat, n_hidden_feat, n_output_feat, g.etypes)
elif vModel == 'SAGE':
    model = ModelHSAGE(n_input_feat, n_hidden_feat, n_output_feat, g.etypes)

opt = torch.optim.Adam(model.parameters())
for epoch in range(n_epochs):
    
    #forward
    neg_src_train, neg_dst_train, negative_graph_train = construct_negative_graph(g_train, k, ('Food', 'isSubstitutedBy', 'Food'))
    h_train, pos_score_train, neg_score_train = model(g_train, negative_graph_train, node_features, ('Food', 'isSubstitutedBy', 'Food'))
    
    #loss
    loss = compute_loss(pos_score_train, neg_score_train)
    
    #backward
    opt.zero_grad()
    loss.backward()
    opt.step()
    
    if epoch % 100 == 0:
        print(f'In epoch {epoch}, loss = {loss.item():.4f}')

In epoch 0, loss = 5.2723
In epoch 100, loss = 0.8152
In epoch 200, loss = 0.4927


# 3) Evaluate Model (based on positive edges that are in the test set)

https://github.com/dglai/WWW20-Hands-on-Tutorial/blob/master/basic_tasks/3_link_predict.ipynb

In [220]:
with torch.no_grad():
    model.eval()
    neg_src_test, neg_dst_test, negative_graph_test = construct_negative_graph(g_test, k, ('Food', 'isSubstitutedBy', 'Food'))
    h_test, pos_score_test, neg_score_test = model(g_test, negative_graph_test, node_features, ('Food', 'isSubstitutedBy', 'Food'))
    loss = compute_loss(pos_score_test, neg_score_test)

In [221]:
# get postive test edges
test_pos_u, test_pos_v = g_test['isSubstitutedBy'].edges()

In [222]:
# get predictions
predictions_pos = torch.as_tensor(pos_score_test.reshape(pos_score_test.shape[0]))
predictions_pos = torch.sigmoid(predictions_pos)

In [223]:
# get y_pred
y_pred_pos = []
for pred in predictions_pos:
    if pred >= 0.5:
        y_pred_pos.append(1)
    else:
        y_pred_pos.append(0)

In [224]:
# get y_true 
y_true_pos = torch.ones(len(pos_score_test))

In [225]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [226]:
print('Accuracy: %.4f' % accuracy_score(y_true_pos, y_pred_pos))

Accuracy: 0.5289


In [227]:
print('Precision: %.4f' % precision_score(y_true_pos, y_pred_pos))

Precision: 1.0000


In [228]:
print('Recall: %.4f' % recall_score(y_true_pos, y_pred_pos))

Recall: 0.5289


In [229]:
print('F-1: %.4f' % f1_score(y_true_pos, y_pred_pos))

F-1: 0.6919


In [230]:
foods_2_test = pd.DataFrame()
foods_2_test['id'] = le_food.inverse_transform(test_pos_u)
foods_2_test.to_csv('../Output/foods_2_test.csv')

# 4) Save 'Food' Embeddings to RDF2Vec Format

In [231]:
foods = le_food.inverse_transform(g_test.nodes('Food')).tolist()

In [232]:
food_embeddings = dict(zip(foods, h_test['Food']))

In [251]:
fw = open('../Output/food_embeddings.txt','w')
fw.write(str(len(foods))+' '+str(len(h_test['Food'][0]))+'\n')
for food in foods:
    fw.write(food+' ')
    for i in range(len(h_test['Food'][0])):
        value = str(food_embeddings[food][i].item()).strip()
        fw.write(value+' ')
    fw.write('\n')

# 5) Predict new links (based on negative edges that are in the train and test sets)

In [273]:
df = pd.DataFrame()
df['u'] = torch.cat([torch.as_tensor(neg_src_train), torch.as_tensor(neg_src_test)])
df['v'] = torch.cat([torch.as_tensor(neg_dst_train), torch.as_tensor(neg_dst_test)])
df['Food id'] = le_food.inverse_transform(df['u'])
df['Substitution id'] = le_food.inverse_transform(df['v'])
df['Scores'] = torch.cat([torch.as_tensor(neg_score_train), torch.as_tensor(neg_score_test)]).detach().numpy()
df['Scores (Sigmoid)'] = torch.sigmoid(torch.cat([torch.as_tensor(neg_score_train), torch.as_tensor(neg_score_test)])).detach().numpy()

**Get Food Labels**

In [274]:
food_labels = pd.read_excel('../Input Data/ABBREV.xlsx', sheet_name='ABBREV')
food_labels.NDB_No = 'http://idea.rpi.edu/heals/kb/usda#' + food_labels.NDB_No.astype(str).str.rjust(5,'0')
food_2_label = dict()

for i, row in food_labels.iterrows():
    food = row['NDB_No']
    label = row['Shrt_Desc']
    food_2_label[food] = label

**Get Nutri-Values + Nutri-Scores**

In [275]:
nutri_scores = pd.read_csv('../Output/nutri_scores.csv')
food_2_score = dict()

for i, row in nutri_scores.iterrows():
    food = row['NDB_No']
    score = row['nutri_values']
    food_2_score[food] = score

**Get Food Categories**

In [276]:
food_cat = pd.read_csv('../Input Data/food_category.csv')
food_cat['NDB_No'] = food_cat['NDB_No'].astype(str).str.rjust(5,'0')
food_2_cat = dict()

for i, row in food_cat.iterrows():
    food = 'http://idea.rpi.edu/heals/kb/usda#' + row['NDB_No']
    cat = row['FdGrp_Desc']
    food_2_cat[food] = cat

**Add Food Labels, Nutri-Values + Nutri-Scores, Food Categories**

In [277]:
food_labels = []
subs_labels = []
food_scores = []
subs_scores = []
food_cat = []
subs_cat = []

for i, row in df.iterrows():
    if(row['Food id'] in food_2_label.keys() and row['Substitution id'] in food_2_label.keys()):
        food_labels.append(food_2_label[row['Food id']])
        subs_labels.append(food_2_label[row['Substitution id']])
    else:
        food_labels.append('not found')
        subs_labels.append('not found')
    if(row['Food id'] in food_2_score.keys() and row['Substitution id'] in food_2_score.keys()):
        food_scores.append(food_2_score[row['Food id']])
        subs_scores.append(food_2_score[row['Substitution id']])
    else:
        food_scores.append(999)
        subs_scores.append(999)
    if(row['Food id'] in food_2_cat.keys() and row['Substitution id'] in food_2_cat.keys()):
        food_cat.append(food_2_cat[row['Food id']])
        subs_cat.append(food_2_cat[row['Substitution id']])
    else:
        food_cat.append('not found')
        subs_cat.append('not found')
        
df['Food label'] = food_labels
df['Substitution label'] = subs_labels
df['Food Nutri-Value'] = food_scores
df['Substitution Nutri-Value'] = subs_scores
df['Food Category'] = food_cat
df['Substitution Category'] = subs_cat

**Filter out all substitutes which can't be found, and which have a higher Nutri-Value than their query food**

In [278]:
len(df)

18410

In [279]:
df = df[df['Food label'] != 'not found']
df = df[df['Food Category'] != 'not found']

In [280]:
len(df)

16536

In [281]:
df = df[df['Food Nutri-Value'] >= df['Substitution Nutri-Value']]

In [282]:
len(df)

7150

**Get final dataset (by filtering out substitutes which are not in the same food category than their query food), which will be labelled by our Nutri-Scholars**

In [331]:
len(np.unique(df['Food id']))

328

In [332]:
df_results = pd.DataFrame()

for food in np.unique(df['Food id']):
    if len(df[(df['Food id'] == food) & (df['Food Category'] == df['Substitution Category'])]) >= 2:
        subs_a = df[(df['Food id'] == food) & (df['Food Category'] == df['Substitution Category'])].sort_values(by=['Scores (Sigmoid)', 'Substitution Nutri-Value'], ascending=[False, True]).head(1)
        df_results = df_results.append(subs_a)
        subs_b = df[(df['Food id'] == food) & (df['Food Category'] == df['Substitution Category'])].sort_values(by=['Substitution Nutri-Value', 'Scores (Sigmoid)'], ascending=[True, False]).head(1)
        if subs_a['Substitution id'].iloc[0] != subs_b['Substitution id'].iloc[0]:
            df_results = df_results.append(subs_b)
        else: 
            df_results = df_results.append(df[(df['Food id'] == food) & (df['Food Category'] == df['Substitution Category'])].sort_values(by=['Substitution Nutri-Value', 'Scores (Sigmoid)'], ascending=[True, False]).head(2))

In [333]:
df_results = df_results.drop_duplicates()

In [334]:
len(np.unique(df_results['Food id']))

77

In [335]:
df_results = df_results[['Food label', 'Substitution label']]

In [336]:
df_results.to_csv('../Output/results.csv')